In [1]:
%load_ext cppimport.magic

In [2]:
%%cppimport?

Docstring:
::

  %cppimport [-v] cpp_module

Build and import C++/C module from ``%%ccpimport`` cell

The content of the cell is written to a file in the
directory ``$IPYTHONDIR/cppimport/<random>/<hash>/`` using
a dirname with the hash of the code, flags and configuration
data. This file is then compiled. The resulting module is
imported.

Usage
=====
Prepend ``%%cppimport`` to your C++/C code in a cell:

%%cppimport module.cpp
// put your code here.

positional arguments:
  cpp_module       Module C/C++ source file.

options:
  -v, --verbosity  Increase output verbosity.
File:      ~/soft/valq/Yandex-Q/fortran_magic/cppimport/cppimport/magic.py

In [3]:
%cppimport_config?

Docstring:
::

  %cppimport_config [-v] [--clean-cache] [--defaults]

options:
  -v, --verbosity  Increase output verbosity.
  --clean-cache    Clean ``cppimport.magic`` build cache.
  --defaults       Delete custom configuration and back to default.
File:      ~/soft/valq/Yandex-Q/fortran_magic/cppimport/cppimport/magic.py

In [4]:
%cppimport_config -v

INF:cppimport.magic:New default arguments for %%cppimport: -v


In [5]:
%cppimport_config --clean-cache

INF:cppimport.magic:Clean cache: /Users/leo/.cache/ipython/cppimport/c9f62062


In [6]:
first_replay_test = len(In)

In [7]:
%%cppimport cpp14module.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
/*<%
import pybind11
cfg['compiler_args'] = ['-std=c++14', '-DOFFSET=0']
cfg['include_dirs'] = [pybind11.get_include(), pybind11.get_include(True)]
%>*/
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int j) {
    return i + j + OFFSET;
}

PYBIND11_MODULE(cpp14module, m) {
    m.def("add", &add);
}

INF:cppimport.magic:C/C++ objects: cpp14module.add


In [8]:
assert 3+5 == cpp14module.add(3, 5)

In [9]:
%%cppimport raw_extension.c
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/raw_extension.c
#include <Python.h>

#if PY_MAJOR_VERSION >= 3
    #define MOD_INIT(name) PyMODINIT_FUNC PyInit_##name(void)
    #define MOD_DEF(ob, name, doc, methods) \
        static struct PyModuleDef moduledef = { \
            PyModuleDef_HEAD_INIT, name, doc, -1, methods, }; \
        ob = PyModule_Create(&moduledef);
    #define MOD_SUCCESS_VAL(val) val
#else
    #define MOD_INIT(name) PyMODINIT_FUNC init##name(void)
    #define MOD_DEF(ob, name, doc, methods) \
        ob = Py_InitModule3(name, methods, doc);
    #define MOD_SUCCESS_VAL(val)
#endif

static PyObject* add(PyObject* self, PyObject* args) {
    int a, b;
    //int class = 1;
    if (!PyArg_ParseTuple(args, "ii", &a, &b)) {
        return NULL;
    }
    return Py_BuildValue("i", a + b);
}

static PyMethodDef methods[] = {
    {"add", add, METH_VARARGS,
     "add(a: np.int32, b: np.int32) -> np.int32)"},
    {NULL}
};

MOD_INIT(raw_extension) {
    PyObject* m;
    MOD_DEF(m, "raw_extension", "", methods)
    return MOD_SUCCESS_VAL(m);
}

INF:cppimport.magic:C/C++ objects: raw_extension.add


In [10]:
assert 3+5 == raw_extension.add(3, 5)

In [11]:
%%cppimport operators.cpp
// Source:
// https://github.com/TNG/boost-python-examples/blob/main/07-Operators/operators.cpp
/*<%
import sys
cfg['libraries'] = ['boost_python' +
                    str(sys.version_info.major) + str(sys.version_info.minor)]
%>*/
#include <sstream>
#include <string>

class NumberLike {
public:
    NumberLike(int n = 0) : mN(n) {}
    NumberLike& operator+= (int i) {
        mN += i;
        return *this;
    }
    std::string str() const {
        std::stringstream s;
        s << mN;
        return s.str();
    }
    std::string repr() const {
        std::stringstream s;
        s << "NumberLike("<< mN << ")";
        return s.str();
    }
private:
    int mN;
};

NumberLike operator+(NumberLike n, int i) {    n += i;
    return n;
};

#include <boost/python.hpp>
using namespace boost::python;

BOOST_PYTHON_MODULE(operators) {
    class_<NumberLike>("NumberLike")
        .def(init< optional<int> >())
        .def(self + int())
        .def("__str__", &NumberLike::str)
        .def("__repr__", &NumberLike::repr)
    ;
}

INF:cppimport.magic:C/C++ objects: operators.NumberLike


In [12]:
n = operators.NumberLike(7)
m = n + 2
assert str(m) == '9'
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == 'NumberLike(1)'
print(n, m, str(m), n0, m0, repr(m0))

7 9 9 0 1 NumberLike(1)


In [13]:
%%writefile .temp_test_extra_sources1.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}

Writing .temp_test_extra_sources1.cpp


In [14]:
%%cppimport extra_sources.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] = True
%>*/
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}

INF:cppimport.magic:C/C++ objects: extra_sources.square_sum


In [15]:
assert 25 == extra_sources.square_sum(3, 4)

### TODO

Описать py::module_local()

In [16]:
%%writefile .temp_test_thing.h
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/test_cppimport.py
#include <iostream>
struct Thing {
    void cheer() {
        std::cout << "WAHHOOOO" << std::endl;
    }
};
#define THING_DEFINED

Writing .temp_test_thing.h


In [17]:
%%writefile .temp_test_thing2.h
// This file is intentionally left empty.

Writing .temp_test_thing2.h


In [18]:
%%cppimport mymodule.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/mymodule.cpp
/*<%
import os
setup_pybind11(cfg)
cfg['include_dirs'] = [os.path.abspath('.')]
cfg['dependencies'] = [os.path.abspath('.temp_test_thing.h')]
%>*/
#include <pybind11/pybind11.h>
#include ".temp_test_thing.h"
#include ".temp_test_thing2.h"

namespace py = pybind11;

int add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(mymodule, m) {
    m.def("add", &add);
#ifdef THING_DEFINED
    // #pragma message "stuff"
    py::class_<Thing>(m, "Thing", py::module_local())
        .def(py::init<>())
        .def("cheer", &Thing::cheer);
#endif
}

INF:cppimport.magic:C/C++ objects: mymodule.add mymodule.Thing


In [19]:
assert 3+5 == mymodule.add(3, 5)

In [20]:
mymodule.Thing().cheer()

WAHHOOOO


In [21]:
import sys
(mymodule.__name__,
 [k for k in sys.modules if 'mymodule' in k],
 mymodule.__file__)

('mymodule',
 ['_4fa7602336eb6f3e86f8bca119d03777_mymodule'],
 '/Users/leo/.cache/ipython/cppimport/ffeb6cff/4fa7602336eb6f3e86f8bca119d03777/_4fa7602336eb6f3e86f8bca119d03777_mymodule.cpython-311-darwin.so')

In [22]:
last_replay_test = len(In)

In [23]:
import sys
import cppimport

force_assert_fail = -1 #4
for test_case in [("cppimport.settings['force_rebuild'] = False;"
                   "get_ipython().run_line_magic('cppimport_config', '-v');"
                  ),
                  ("cppimport.settings['force_rebuild'] = True;"
                  ),
                  ("get_ipython().run_line_magic('cppimport_config', '-vvv');"
                  )
                 ]:
    print('\033[95m exec( ',
      test_case,
      ' )\033[0m', flush=True)
    exec(test_case)
    for i in range(first_replay_test, last_replay_test-1):
        ii = In[i]
        if force_assert_fail != 0:
            force_assert_fail -= 1
        elif 'assert' in ii:
            ii = re.sub(r'(assert )([0-9])', r'\1 1\2', ii)
            ii = re.sub(r'(assert )([^0-9])', r'\1 " " + \2', ii)
        print('\033[92m exec(\033[0m',
              ii.replace('\\n', '\n').replace("\\'", "'"),
              '\033[92m )\033[0m', flush=True)
        exec(ii)
        sys.stdout.flush()
        sys.stderr.flush()

 exec(  cppimport.settings['force_rebuild'] = False;get_ipython().run_line_magic('cppimport_config', '-v');  )
INF:cppimport.magic:New default arguments for %%cppimport: -v
 exec( get_ipython().run_cell_magic('cppimport', 'cpp14module.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
/*<%
import pybind11
cfg['compiler_args'] = ['-std=c++14', '-DOFFSET=0']
cfg['include_dirs'] = [pybind11.get_include(), pybind11.get_include(True)]
%>*/
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int j) {
    return i + j + OFFSET;
}

PYBIND11_MODULE(cpp14module, m) {
    m.def("add", &add);
}
')  )


 exec( assert 3+5 == cpp14module.add(3, 5)  )
 exec( get_ipython().run_cell_magic('cppimport', 'raw_extension.c', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/raw_extension.c
#include <Python.h>

#if PY_MAJOR_VERSION >= 3
    #define MOD_INIT(name) PyMODINIT_FUNC PyInit_##name(void)
    #define MOD_DEF(ob, name, doc, methods) \\
        static struct PyModuleDef moduledef = { \\
            PyModuleDef_HEAD_INIT, name, doc, -1, methods, }; \\
        ob = PyModule_Create(&moduledef);
    #define MOD_SUCCESS_VAL(val) val
#else
    #define MOD_INIT(name) PyMODINIT_FUNC init##name(void)
    #define MOD_DEF(ob, name, doc, methods) \\
        ob = Py_InitModule3(name, methods, doc);
    #define MOD_SUCCESS_VAL(val)
#endif

static PyObject* add(PyObject* self, PyObject* args) {
    int a, b;
    //int class = 1;
    if (!PyArg_ParseTuple(args, "ii", &a, &b)) {
        return NULL;
    }
    return Py_BuildValue("i", a + b);
}

static PyMethodDef methods[] = {
    

 exec( assert 3+5 == raw_extension.add(3, 5)  )
 exec( get_ipython().run_cell_magic('cppimport', 'operators.cpp', '// Source:
// https://github.com/TNG/boost-python-examples/blob/main/07-Operators/operators.cpp
/*<%
import sys
cfg['libraries'] = ['boost_python' +
                    str(sys.version_info.major) + str(sys.version_info.minor)]
%>*/
#include <sstream>
#include <string>

class NumberLike {
public:
    NumberLike(int n = 0) : mN(n) {}
    NumberLike& operator+= (int i) {
        mN += i;
        return *this;
    }
    std::string str() const {
        std::stringstream s;
        s << mN;
        return s.str();
    }
    std::string repr() const {
        std::stringstream s;
        s << "NumberLike("<< mN << ")";
        return s.str();
    }
private:
    int mN;
};

NumberLike operator+(NumberLike n, int i) {    n += i;
    return n;
};

#include <boost/python.hpp>
using namespace boost::python;

BOOST_PYTHON_MODULE(operators) {
    class_<NumberLike>("NumberLike")
    

 exec( n = operators.NumberLike(7)
m = n + 2
assert str(m) == '9'
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == 'NumberLike(1)'
print(n, m, str(m), n0, m0, repr(m0))  )
7 9 9 0 1 NumberLike(1)
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_extra_sources1.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}
')  )
Overwriting .temp_test_extra_sources1.cpp
 exec( get_ipython().run_cell_magic('cppimport', 'extra_sources.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] = True
%>*/
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}
')  )


 exec( assert 25 == extra_sources.square_sum(3, 4)  )
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_thing.h', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/test_cppimport.py
#include <iostream>
struct Thing {
    void cheer() {
        std::cout << "WAHHOOOO" << std::endl;
    }
};
#define THING_DEFINED
')  )
Overwriting .temp_test_thing.h
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_thing2.h', '// This file is intentionally left empty.
')  )
Overwriting .temp_test_thing2.h
 exec( get_ipython().run_cell_magic('cppimport', 'mymodule.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/mymodule.cpp
/*<%
import os
setup_pybind11(cfg)
cfg['include_dirs'] = [os.path.abspath('.')]
cfg['dependencies'] = [os.path.abspath('.temp_test_thing.h')]
%>*/
#include <pybind11/pybind11.h>
#include ".temp_test_thing.h"
#include ".temp_test_thing2.h"

namespace py = pybind11;

int add(int i, int j) {
    return i + j;
}

P

 exec( assert 3+5 == mymodule.add(3, 5)  )
 exec( mymodule.Thing().cheer()  )
WAHHOOOO
 exec( import sys
(mymodule.__name__,
 [k for k in sys.modules if 'mymodule' in k],
 mymodule.__file__)  )
 exec(  cppimport.settings['force_rebuild'] = True;  )
 exec( get_ipython().run_cell_magic('cppimport', 'cpp14module.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
/*<%
import pybind11
cfg['compiler_args'] = ['-std=c++14', '-DOFFSET=0']
cfg['include_dirs'] = [pybind11.get_include(), pybind11.get_include(True)]
%>*/
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int j) {
    return i + j + OFFSET;
}

PYBIND11_MODULE(cpp14module, m) {
    m.def("add", &add);
}
')  )
INF:cppimport.magic:C/C++ objects: cpp14module.add
 exec( assert 3+5 == cpp14module.add(3, 5)  )
 exec( get_ipython().run_cell_magic('cppimport', 'raw_extension.c', '// Source:
// https://github.com/tbenthompson/cppimpor

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for NumberLike already registered; second conversion method ignored.


 exec( n = operators.NumberLike(7)
m = n + 2
assert str(m) == '9'
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == 'NumberLike(1)'
print(n, m, str(m), n0, m0, repr(m0))  )
7 9 9 0 1 NumberLike(1)
 exec( get_ipython().run_cell_magic('writefile', '.temp_test_extra_sources1.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}
')  )
Overwriting .temp_test_extra_sources1.cpp
 exec( get_ipython().run_cell_magic('cppimport', 'extra_sources.cpp', '// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] = True
%>*/
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}
')  )
INF:cppimport.magic:C/C++ objects: ext

In [24]:
2+2

4